<a href="https://colab.research.google.com/github/HuseynRajabofff/HuseynRajabofff.github.io/blob/main/AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pyTelegramBotAPI numpy Pillow keras==2.15 tensorflow==2.15


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.3/475.3 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.5/270.5 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.17.2
    Uninstalling wrapt-1.17.2:
      Successfully uninstalled wrapt-1.17.2
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
  Attempting uninstall: keras
    Found existing instal

In [ ]:
TOKEN = '7578316826:AAH10gCb_fLise7Xch6OpR3TVcR2DKOR5wI'

In [ ]:
from google.colab import drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import telebot
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

# ✅ Step 1: Define Correct Model Paths
MODEL_PATH = "/content/keras_model.h5"
LABELS_PATH = "/content/labels.txt"

# ✅ Step 2: Load Model & Labels
try:
    model = load_model(MODEL_PATH)
    print("✅ Model loaded successfully!")
except Exception as e:
    print("❌ Error loading model:", e)
    model = None  # Avoid crashes if model fails to load

# Load class labels
try:
    with open(LABELS_PATH, "r") as f:
        class_names = f.read().splitlines()
    print("✅ Labels loaded successfully!")
except FileNotFoundError:
    print(f"❌ labels.txt not found at {LABELS_PATH}. Please check the path and ensure the file exists.")
    class_names = []

# ✅ Step 3: Set Up Telegram Bot
TOKEN = '7578316826:AAH10gCb_fLise7Xch6OpR3TVcR2DKOR5wI'
bot = telebot.TeleBot(TOKEN)

# Ensure temp folder exists
if not os.path.exists("temp"):
    os.makedirs("temp")

@bot.message_handler(commands=['start', 'help'])
def send_welcome(message):
    bot.send_message(message.chat.id, "Привет! Отправь мне фото машины, и я скажу, что это за бренд 🚗")

@bot.message_handler(content_types=['photo'])
def handle_photo(message):
    """Receives a photo, saves it, and predicts the car brand."""
    if not message.photo:
        return bot.send_message(message.chat.id, 'Фотография не найдена')

    # Get file info
    file_info = bot.get_file(message.photo[-1].file_id)
    file_name = file_info.file_path.split('/')[-1]
    file_path = f"temp/{file_name}"

    # Download the image
    downloaded_file = bot.download_file(file_info.file_path)
    with open(file_path, 'wb') as new_file:
        new_file.write(downloaded_file)

    # Predict car brand
    if model and class_names:
        car_brand, confidence = predict_car(file_path)
        bot.send_message(message.chat.id, f"Я думаю, это **{car_brand}** с вероятностью {confidence:.2f}% 🚗")
    else:
        bot.send_message(message.chat.id, "❌ Ошибка: модель не загружена или отсутствуют метки!")

    # Delete temp file to save space
    os.remove(file_path)

def predict_car(img_path):
    """Loads an image, processes it, and predicts the car brand."""
    img = image.load_img(img_path, target_size=(224, 224))  # Resize to match model input
    img_array = image.img_to_array(img) / 255.0  # Normalize
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions[0])  # Get highest probability index
    confidence = np.max(predictions[0]) * 100  # Convert to percentage

    return class_names[predicted_class], confidence

# ✅ Step 4: Start the Bot
print("🤖 Bot is running...")
bot.infinity_polling()


✅ Model loaded successfully!
✅ Labels loaded successfully!
🤖 Bot is running...
1/1 [==============================] - 1s 952ms/step


2025-03-08 09:11:10,982 (__init__.py:1115 MainThread) ERROR - TeleBot: "Infinity polling exception: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))"
ERROR:TeleBot:Infinity polling exception: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
2025-03-08 09:11:10,987 (__init__.py:1117 MainThread) ERROR - TeleBot: "Exception traceback:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py", line 787, in urlopen
    response = self._make_request(
               ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py", line 534, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connection.py", line 516, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/p

In [ ]:
from keras.models import load_model  # TensorFlow is required for Keras to work
from PIL import Image, ImageOps  # Install pillow instead of PIL
import numpy as np

def get_class(image_path, model_path="gtm_model/keras_model.h5", labels_path="gtm_model/labels.txt"):
    # Disable scientific notation for clarity
    np.set_printoptions(suppress=True)

    # Load the model
    model = load_model(model_path, compile=False)

    # Load the labels
    class_names = open(labels_path, "r", encoding="utf-8").readlines()

    # Create the array of the right shape to feed into the keras model
    # The 'length' or number of images you can put into the array is
    # determined by the first position in the shape tuple, in this case 1
    data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

    # Replace this with the path to your image
    image = Image.open(image_path).convert("RGB")

    # resizing the image to be at least 224x224 and then cropping from the center
    size = (224, 224)
    image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)

    # turn the image into a numpy array
    image_array = np.asarray(image)

    # Normalize the image
    normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

    # Load the image into the array
    data[0] = normalized_image_array

    # Predicts the model
    prediction = model.predict(data)
    index = np.argmax(prediction)
    class_name = class_names[index]
    confidence_score = prediction[0][index]

    # Return prediction and confidence score
    return class_name[2:len(class_name)-1], round(confidence_score*100, 1)
